# Proyecto Computer Vision End-to-End

## Proyecto DEMO para Meetup CaliSharp

![Demo](images/demo.png)

# Paso 1. Idea

![Demo](images/idea.png)

# Paso 2. Anotaciones

## 2.1. Obtener datos

- **Dataset:** [COCO Dataset](https://cocodataset.org/)
- **Descarga de imágenes:** Se hace uso de la librería [FalconCV](https://github.com/haruiz/FalconCV) para realizar la descarga de las imágenes y las anotaciones

In [1]:
import os
from falconcv.ds import Coco
from falconcv.util import FileUtil, ImageUtil, VIUtil, ColorUtil

def download_data(labels_map, color_palette, n_images, batch_size, split, task, output_folder):
    try:
        # creating dataset
        dataset = Coco(v=2017)
        dataset.setup(split=split, task=task)
        os.makedirs(output_folder, exist_ok=True)
        FileUtil.clear_folder(output_folder)
        for batch_images in dataset.fetch(
                n=n_images,
                labels=list(labels_map.keys()),
                batch_size=batch_size):
            for img in batch_images:
                img.export(output_folder, labels_map, color_palette)
    except Exception as ex:
        print(f"Error descargando dataset: {ex}")
            
labels_map = {
    "airplane": 1,
    "train": 2
}
color_palette = ColorUtil.color_palette(n=len(labels_map))

# download train images
download_data(labels_map=labels_map, color_palette=color_palette, n_images=500, 
              batch_size=250, split="train", task="segmentation", output_folder="data/train")
# download validation images
download_data(labels_map=labels_map, color_palette=color_palette, n_images=200, 
              batch_size=100, split="validation", task="segmentation", output_folder="data/val")

INFO     Downloading Coco dataset dependencies, it can take a few minutes
INFO     Download dependencies done


loading annotations into memory...
Done (t=9.48s)
creating index...
index created!


INFO     downloading images for : airplane
INFO     downloading batch 1/2
INFO     downloading batch 2/2
INFO     downloading images for : train
INFO     downloading batch 1/2
INFO     downloading batch 2/2
INFO     Downloading Coco dataset dependencies, it can take a few minutes
INFO     Download dependencies done


loading annotations into memory...
Done (t=0.42s)
creating index...
index created!


INFO     downloading images for : airplane
INFO     downloading batch 1/1
INFO     downloading images for : train
INFO     downloading batch 1/2
INFO     downloading batch 2/2


## 2.2. Realizar anotaciones

- **Info:** Usando datasets como OpenImages y COCO no necesitamos realizar el proceso de anotaciones, ya que estos datasets cuentan con las anotaciones contruidas y son de propósito general
- **Visualización/Ajuste anotaciones:** Se hace uso de la herramienta [CVStudio](https://github.com/haruiz/CvStudio)

![CVStudio](images/cv-studio.png)

# Paso 3. Entrenamiento

Se hace uso de la librería [FalconCV](https://github.com/haruiz/FalconCV) para realizar el entrenamiento del modelo

## 3.1. Seleccionar modelo del Model Zoo

In [2]:
from falconcv.models.tf import ModelZoo

ModelZoo.print_available_models(arch="mask")

DEBUG    Installing Api
DEBUG    Api installation done


*** TensorFlow Detection Model Zoo ***
  mask_rcnn_inception_resnet_v2_atrous_coco
  mask_rcnn_inception_v2_coco
  mask_rcnn_resnet101_atrous_coco
  mask_rcnn_resnet50_atrous_coco


## 3.2. Entrenar modelo

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from falconcv.models import ModelBuilder

def train_and_freeze_model(model_name, images_folder, out_folder, labels_map, epochs=5000):
    try:
        config = {
            "model": model_name,
            "images_folder": images_folder,
            "output_folder": out_folder,
            "labels_map": labels_map,
        }
        with ModelBuilder.build(config=config) as model:
            model.train(epochs=epochs, val_split=0.3, clear_folder=True)
            model.freeze(epochs)
    except Exception as ex:
        raise Exception(f"Error training the model {ex}") from ex

images_folder = "data/train/"
model_folder = "model/tf"

train_and_freeze_model("mask_rcnn_inception_v2_coco", images_folder, model_folder, labels_map)

ERROR    Error loading the model:  <class 'Exception'>, Error training the model : 'Airplane'


Exception: Error training the model Error training the model : 'Airplane'